<a href="https://colab.research.google.com/github/Kerriea-star/Tensorflow-with-Sructured-Data/blob/main/TensorFlow_with_Tabular_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to classify structured data, such as tabular data, using a simplified version of the PetFinder dataset from a Kaggle competition stored in a CSV file.

We will use Keras to define the model, the Keras preprocessing layers as a bridge to map from columns in a CSV file to features used to train the model. The goal is to predict if a pet will be adopted.

This tutorial contains complete code for:


*   Loading a CSV file into a DataFrame using pandas
*   Building an input pipline to batch and shuffle the rows using tf.data.
*   Mapping from columns in the CSV file to features used to train the model with the Keras preprocessing layers.
*   Building, training, and evaluating a model using the Keras built-in methods.



In [7]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [8]:
tf.__version__

'2.12.0'